In [2]:
# import libraries
import pandas as pd
import numpy as np
import os

TRAIN_CSV = 'Train.csv'
VALID_CSV = 'Valid.csv'
MODEL_NAME = 'best_model.h5'
data_path = '/content/drive/MyDrive/Colab Notebooks/chatbot-flask-simple/data/movie_reviews'

training = os.path.join(data_path, TRAIN_CSV)
validation = os.path.join(data_path, VALID_CSV)
model_path = os.path.join(data_path, MODEL_NAME)
# reading csv files
train = pd.read_csv(training)
valid = pd.read_csv(validation)

#train_test split
x_tr, y_tr = train['text'].values, train['label'].values
x_val, y_val = valid['text'].values, valid['label'].values

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#Tokenize the sentences
tokenizer = Tokenizer()
#preparing vocabulary
tokenizer.fit_on_texts(list(x_tr))

# converting text into integer sequences
x_tr_seq = tokenizer.texts_to_sequences(x_tr)
x_val_seq = tokenizer.texts_to_sequences(x_val)

# padding to prepare sequences of same length
x_tr_seq = pad_sequences(x_tr_seq, maxlen=100)
x_val_seq = pad_sequences(x_val_seq, maxlen=100)

size_of_vocabulary = len(tokenizer.word_index) + 1 #+1 for padding
print(size_of_vocabulary)

# build two different NLP models of the same architecture.  The first learns
# embeddings from scratch the second uses pretrained word embeddings
from keras.models import *
from keras.layers import *
from keras.callbacks import *

model = Sequential()

#embedding layer
model.add(Embedding(size_of_vocabulary,300,input_length=100,trainable=True))

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

#Add loss function, metrics, optimizer
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])

#addingcallbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=)
mc = ModelCheckpoint(model_path, monitor='val_acc', mode='max', 
                     save_best_only=True, verbose=1)

#Print summary of model
print(model.summary())

history = model.fit(np.array(x_tr_seq),np.array(y_tr),batch_size=128,epochs=10,
                    validation_data=(np.array(x_val_seq),np.array(y_val)),
                    verbose=1,callbacks=[es,mc])

#loading best model
from keras.models import load_model
model = load_model(model_path)

# evaluation
_,val_acc = model.evaluate(x_val_seq,y_val,batch_size=128)
print(val_acc)

112204
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 300)          33661200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 128)          219648    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 33,889,169
Trainable params: 33,889,169
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
313/313 [==============================] - 8s 21ms/step - loss: 0.4065 - acc: 0.8058 

Build version II using GloVe pretrained word embeddings. Let's load the GloVe embeddings into our environment. 

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open